### Flip and blur the test set images


In [48]:
import numpy as np
import keras
from keras.datasets import mnist 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [49]:
def resizing(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            train_data.append(resized_img)
     return train_data

def flip_ver(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            resized_img = cv2.flip(resized_img, 0)
            train_data.append(resized_img)
     return train_data

def flip_hor(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            resized_img = cv2.flip(resized_img, 1)
            train_data.append(resized_img)
     return train_data
newX_train = np.array(resizing(X_train))
verX_test = np.array(flip_ver(X_test))
horX_test = np.array(flip_hor(X_test))

In [50]:
from keras import backend
if keras.backend.image_data_format() == 'channels_first':
  newX_train = newX_train.reshape(len(newX_train), 1, 32, 32)
  verX_test = verX_test.reshape(len(verX_test), 1, 32, 32)
  horX_test = horX_test.reshape(len(horX_test), 1, 32, 32)
  input_shape = (1, 32, 32)
else:
  newX_train = newX_train.reshape(len(newX_train), 32, 32, 1)
  verX_test = verX_test.reshape(len(verX_test), 32, 32, 1)
  horX_test = horX_test.reshape(len(horX_test), 32, 32, 1)
  input_shape = (32, 32, 1)

In [51]:
from keras.utils import np_utils
def nor(dataset):
  dataset = dataset.astype('float32')
  dataset /= 255
  return dataset
newX_train = nor(newX_train)
verX_test = nor(verX_test)
horX_test = nor(horX_test)
Y_train = keras.utils.np_utils.to_categorical(Y_train, 10)
Y_test = keras.utils.np_utils.to_categorical(Y_test, 10)

In [52]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten 
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Flatten()) 
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 32, 32, 64)        640       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 8, 8, 128)        0         
 g2D)                                                            
                                                                 
 conv2d_42 (Conv2D)          (None, 8, 8, 256)         295168    
                                                                 
 conv2d_43 (Conv2D)          (None, 8, 8, 256)        

In [53]:
from tensorflow.keras.optimizers import SGD
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [11]:
# history = model.fit(newX_train[:6000], Y_train[:6000], batch_size=32, validation_data=(verX_test,Y_test), epochs=10, verbose=1)
# model.evaluate(verX_test, Y_test, batch_size=32)[1]

Epoch 1/10
188/188 [==============================] - 468s 2s/step - loss: 2.0570 - accuracy: 0.2913 - val_loss: 4.1283 - val_accuracy: 0.1900
Epoch 2/10
188/188 [==============================] - 465s 2s/step - loss: 0.6647 - accuracy: 0.7685 - val_loss: 4.7797 - val_accuracy: 0.2714
Epoch 3/10
188/188 [==============================] - 461s 2s/step - loss: 0.2288 - accuracy: 0.9338 - val_loss: 3.4100 - val_accuracy: 0.3175
Epoch 4/10
188/188 [==============================] - 435s 2s/step - loss: 0.1393 - accuracy: 0.9603 - val_loss: 3.6378 - val_accuracy: 0.3225
Epoch 5/10
188/188 [==============================] - 430s 2s/step - loss: 0.0968 - accuracy: 0.9710 - val_loss: 4.5351 - val_accuracy: 0.2511
Epoch 6/10
188/188 [==============================] - 432s 2s/step - loss: 0.0731 - accuracy: 0.9773 - val_loss: 4.7789 - val_accuracy: 0.3490
Epoch 7/10
188/188 [==============================] - 432s 2s/step - loss: 0.0630 - accuracy: 0.9802 - val_loss: 4.0217 - val_accuracy: 0.3306

0.3249000012874603

##### The vertical flip test accuracy is 0.3249, which is greatly reduced compared to previous one.


In [55]:
history = model.fit(newX_train[:6000], Y_train[:6000], batch_size=32, validation_data=(horX_test,Y_test), epochs=10, verbose=1)
model.evaluate(horX_test, Y_test, batch_size=32)[1]

Epoch 1/10
188/188 [==============================] - 443s 2s/step - loss: 2.2923 - accuracy: 0.1392 - val_loss: 2.2484 - val_accuracy: 0.2813
Epoch 2/10
188/188 [==============================] - 444s 2s/step - loss: 1.4601 - accuracy: 0.4870 - val_loss: 3.5541 - val_accuracy: 0.2884
Epoch 3/10
188/188 [==============================] - 443s 2s/step - loss: 0.3015 - accuracy: 0.9090 - val_loss: 3.5167 - val_accuracy: 0.3417
Epoch 4/10
188/188 [==============================] - 444s 2s/step - loss: 0.1533 - accuracy: 0.9543 - val_loss: 4.2139 - val_accuracy: 0.3489
Epoch 5/10
188/188 [==============================] - 441s 2s/step - loss: 0.1030 - accuracy: 0.9692 - val_loss: 3.5716 - val_accuracy: 0.4475
Epoch 6/10
188/188 [==============================] - 439s 2s/step - loss: 0.0794 - accuracy: 0.9775 - val_loss: 3.8853 - val_accuracy: 0.3889
Epoch 7/10
188/188 [==============================] - 440s 2s/step - loss: 0.0607 - accuracy: 0.9813 - val_loss: 3.7742 - val_accuracy: 0.3730

0.33219999074935913

##### The vertical flip test accuracy is 0.3322, which is greatly reduced compared to previous one.

### Add standard Gaussian noise

In [56]:
import numpy as np
import keras
from keras.datasets import mnist 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [57]:
def resizing(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            train_data.append(resized_img)
     return train_data

newX_train = np.array(resizing(X_train))
newX_test = np.array(resizing(X_test))

In [58]:
from keras import backend
if keras.backend.image_data_format() == 'channels_first':
  newX_train = newX_train.reshape(len(newX_train), 1, 32, 32)
  newX_test = newX_test.reshape(len(newX_test), 1, 32, 32)
  input_shape = (1, 32, 32)
else:
  newX_train = newX_train.reshape(len(newX_train), 32, 32, 1)
  newX_test = newX_test.reshape(len(newX_test), 32, 32, 1)
  horX_test = horX_test.reshape(len(horX_test), 32, 32, 1)
  input_shape = (32, 32, 1)

In [59]:
from keras.utils import np_utils
newX_train = newX_train.astype('float32')
newX_train /= 255
newX_test = newX_test.astype('float32')
newX_test /= 255
Y_train = keras.utils.np_utils.to_categorical(Y_train, 10)
Y_test = keras.utils.np_utils.to_categorical(Y_test, 10)

In [61]:
from tensorflow.keras.layers import GaussianNoise
var = [0.01, 0.1, 1]
for i in var:
  sample = GaussianNoise(i, dtype=tf.float64)
  noisey = sample(newX_test,training=True) 
  model = Sequential()
  model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Flatten()) 
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  sgd = SGD(learning_rate=0.01, decay=1e-3, momentum=0.9, nesterov=True)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  history = model.fit(newX_train[:6000], Y_train[:6000], batch_size=32, validation_data=(newX_test,Y_test), epochs=10, verbose=1)
  acc = model.evaluate(newX_test, Y_test, batch_size=32)[1]
  print("Test accuracy when variance is " + i + "is " + acc)
print("Test accuracy when variance is " + i + "is " + acc)

Epoch 1/10
188/188 [==============================] - 454s 2s/step - loss: 2.3006 - accuracy: 0.1075 - val_loss: 2.3000 - val_accuracy: 0.1028
Epoch 2/10
188/188 [==============================] - 447s 2s/step - loss: 2.2759 - accuracy: 0.1885 - val_loss: 2.1143 - val_accuracy: 0.2957
Epoch 3/10
188/188 [==============================] - 443s 2s/step - loss: 1.0821 - accuracy: 0.6157 - val_loss: 0.5441 - val_accuracy: 0.8251
Epoch 4/10
188/188 [==============================] - 445s 2s/step - loss: 0.3144 - accuracy: 0.9072 - val_loss: 0.2282 - val_accuracy: 0.9289
Epoch 5/10
188/188 [==============================] - 445s 2s/step - loss: 0.1580 - accuracy: 0.9517 - val_loss: 0.2303 - val_accuracy: 0.9291
Epoch 6/10
188/188 [==============================] - 442s 2s/step - loss: 0.1138 - accuracy: 0.9662 - val_loss: 0.1257 - val_accuracy: 0.9628
Epoch 7/10
188/188 [==============================] - 444s 2s/step - loss: 0.0746 - accuracy: 0.9787 - val_loss: 0.1072 - val_accuracy: 0.9680

TypeError: ignored

In [64]:
print("Test accuracy for variance of " + str(i) + " is " + str(acc))

Test accuracy for variance of 0.01 is 0.9684000015258789


In [65]:
from tensorflow.keras.layers import GaussianNoise
var = [0.1, 1]
for i in var:
  sample = GaussianNoise(i, dtype=tf.float64)
  noisey = sample(newX_test,training=True) 
  model = Sequential()
  model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Flatten()) 
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  sgd = SGD(learning_rate=0.01, decay=1e-3, momentum=0.9, nesterov=True)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  history = model.fit(newX_train[:6000], Y_train[:6000], batch_size=32, validation_data=(newX_test[:1000],Y_test[:1000]), epochs=10, verbose=1)
  acc = model.evaluate(newX_test[:1000], Y_test[:1000], batch_size=32)[1]
  print("Test accuracy when variance is " + str(i) + "is " + str(acc))

Epoch 1/10
188/188 [==============================] - 505s 2s/step - loss: 2.3003 - accuracy: 0.1135 - val_loss: 2.2970 - val_accuracy: 0.1028
Epoch 2/10
188/188 [==============================] - 444s 2s/step - loss: 2.2170 - accuracy: 0.2188 - val_loss: 1.6515 - val_accuracy: 0.4580
Epoch 3/10
188/188 [==============================] - 445s 2s/step - loss: 0.7784 - accuracy: 0.7350 - val_loss: 0.3651 - val_accuracy: 0.8851
Epoch 4/10
188/188 [==============================] - 436s 2s/step - loss: 0.2451 - accuracy: 0.9257 - val_loss: 0.1781 - val_accuracy: 0.9431
Epoch 5/10
188/188 [==============================] - 434s 2s/step - loss: 0.1508 - accuracy: 0.9537 - val_loss: 0.1845 - val_accuracy: 0.9452
Epoch 6/10
188/188 [==============================] - 435s 2s/step - loss: 0.1010 - accuracy: 0.9695 - val_loss: 0.1321 - val_accuracy: 0.9603
Epoch 7/10
188/188 [==============================] - 435s 2s/step - loss: 0.0663 - accuracy: 0.9785 - val_loss: 0.1117 - val_accuracy: 0.9660

In [ ]:
# from tensorflow.keras.layers import GaussianNoise
# var = [0.01, 0.1, 1]
# for i in var:
#   sample = GaussianNoise(i, dtype=tf.float64)
#   noisey = sample(newX_test,training=True) 
#   model = Sequential()
#   model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
#   model.add(Conv2D(128, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
#   model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
#   model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
#   model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
#   model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
#   model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
#   model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
#   model.add(Flatten()) 
#   model.add(Dense(4096, activation='relu'))
#   model.add(Dropout(0.5))
#   model.add(Dense(4096, activation='relu'))
#   model.add(Dropout(0.5))
#   model.add(Dense(10, activation='softmax'))
#   sgd = SGD(learning_rate=0.01, decay=1e-3, momentum=0.9, nesterov=True)
#   model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#   history = model.fit(newX_train[:6000], Y_train[:6000], batch_size=32, validation_data=(newX_test[:1000],Y_test[:1000]), epochs=10, verbose=1)
#   acc = model.evaluate(newX_test[:1000], Y_test[:1000], batch_size=32)[1]
#   print("Test accuracy when variance is " + str(i) + "is " + str(acc))

#### Summary

1.   Test Accuracy when Gaussian noise of variance 0.01: 0.9684
2.   Test Accuracy when Gaussian noise of variance 0.1: 0.9672
3.   Test Accuracy when Gaussian noise of variance 1: 0.9686

##### Test accuracy decreases as variance of noise increases, considering that noises change the images and make them quite different from orginal images.


In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('A3E1_3.ipynb')

--2022-03-03 07:53:35--  https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1864 (1.8K) [text/plain]
Saving to: ‘colab_pdf.py’

colab_pdf.py        100%[===================>]   1.82K  --.-KB/s    in 0s      

2022-03-03 07:53:35 (18.6 MB/s) - ‘colab_pdf.py’ saved [1864/1864]

Mounted at /content/drive/




Extracting templates from packages: 100%
